# Introduction to Docker

**Learning Objectives**
  * Build and run Docker containers
  * Pull Docker images from Docker Hub and Google Container Registry
  * Push Docker images to Google Container Registry

## Overview

Docker is an open platform for developing, shipping, and running applications. With Docker, you can separate your applications from your infrastructure and treat your infrastructure like a managed application. Docker helps you ship code faster, test faster, deploy faster, and shorten the cycle between writing code and running code.

Docker does this by combining kernel containerization features with workflows and tooling that helps you manage and deploy your applications.

Docker containers can be directly used in Kubernetes, which allows them to be run in the Kubernetes Engine with ease. After learning the essentials of Docker, you will have the skillset to start developing Kubernetes and containerized applications.

## Basic Docker commands

See what docker images you have. 

In [1]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED         SIZE
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   13 months ago   1.73GB


If this is the first time working with docker you won't have any repositories listed. 

**Note**. If you are running this in an AI Notebook, then you should see a single image `gcr.io/inverting-proxy/agent`. This is the container that is currently running the AI Notebook. 

Let's use `docker run` to pull a docker image called `hello-world` from the public registry. The docker daemon will search for the `hello-world` image, if it doesn't find the image locally, it pulls the image from a public registry called Docker Hub, creates a container from that image, and runs the container for you.

In [2]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:bfea6278a0a267fad2634554f4f0c6f31981eea41c553fdf5a83e95a41d40c38
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https

Now when we look at our docker images we should see `hello-world` there as well.

In [3]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED         SIZE
hello-world                    latest    feb5d9fea6a5   6 months ago    13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   13 months ago   1.73GB


This is the image pulled from the Docker Hub public registry. The Image ID is in `SHA256` hash format—this field specifies the Docker image that's been provisioned. When the docker daemon can't find an image locally, it will by default search the public registry for the image. Let's run the container again:

Now, if we want to run `docker run hello-world` again, it won't have to download from the container registry.

To see all docker containers running, use `docker ps`.

In [4]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS     NAMES
cc9930dc7af4   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   12 minutes ago   Up 10 minutes             proxy-agent


There are no running containers. **Note. If you are running this in at AI Notebook, you'll see one container running.**

The `hello-world` containers you ran previously already exited. In order to see all containers, including ones that have finished executing, run docker `ps -a`:

In [5]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                      PORTS     NAMES
74f7760c502c   hello-world                    "/hello"                 23 seconds ago   Exited (0) 22 seconds ago             awesome_morse
cc9930dc7af4   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   12 minutes ago   Up 10 minutes                         proxy-agent


This shows you the Container ID, a UUID generated by Docker to identify the container, and more metadata about the run. The container Names are also randomly generated but can be specified with docker run --name [container-name] hello-world.

## Build a Docker container

Let's build a Docker image that's based on a simple node application.

**Exercise**

Open the text file called `intro.docker` in the `dockerfiles` folder and complete the TODO there. 

Your dockerfile should have the following steps

 1. use `FROM` to inherit an official Node runtime as the parent image; e.g. node:6
 2. use `WORKDIR` to seet the working directory to /app
 3. use `ADD` to copy the current directory to the container at /app
 4. use `EXPOSE` to make the containers port 80 available to the outside world
 5. use `CMD` to run the command `node ./src/app.js`

This file instructs the Docker daemon on how to build your image.

The initial line specifies the base parent image, which in this case is the official Docker image for node version 6.
In the second, we set the working (current) directory of the container.
In the third, we add the current directory's contents (indicated by the "." ) into the container.
Then we expose the container's port so it can accept connections on that port and finally run the node command to start the application.

Check out the other [Docker command references](https://docs.docker.com/engine/reference/builder/#known-issues-run) to understand what each line does.

We're going to use this Docker container to run a simple node.js app. Have a look at `app.js`. This is a simple HTTP server that listens on port 80 and returns "Hello World."


Now let's build the image. Note again the "`.`", which means current directory so you need to run this command from within the directory that has the Dockerfile.

The `-t` is to name and tag an image with the `name:tag` syntax. The name of the image is `node-app` and the tag is `0.1`. The tag is highly recommended when building Docker images. If you don't specify a tag, the tag will default to latest and it becomes more difficult to distinguish newer images from older ones. Also notice how each line in the Dockerfile above results in intermediate container layers as the image is built.

**Exercise**

Use `docker build` to build the docker image at `dockerfiles/intro.docker`. Tag the image `node-app:0.1`. 

In [13]:
%%bash
docker build -t node-app:0.1 ./  -f ./dockerfiles/intro.docker

Sending build context to Docker daemon  124.4kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> 8944fe56d3aa
Step 3/5 : COPY ./src/ /app
 ---> fe5c276c5549
Step 4/5 : EXPOSE 80
 ---> Running in 65585724ac4c
Removing intermediate container 65585724ac4c
 ---> d5ef84935c98
Step 5/5 : CMD ["node", "app.js"]
 ---> Running in 2a689e03ca31
Removing intermediate container 2a689e03ca31
 ---> 81eb2923eadb
Successfully built 81eb2923eadb
Successfully tagged node-app:0.1


Let's check that the image has been created correctly. 

In [9]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED          SIZE
node-app                       0.1       13fc570556c6   38 seconds ago   884MB
hello-world                    latest    feb5d9fea6a5   6 months ago     13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   13 months ago    1.73GB
node                           6         ab290b853066   2 years ago      884MB


You should see a `node-app` repository that was created only seconds ago. 

Notice `node` is the base image and `node-app` is the image you built. You can't remove `node` without removing `node-app` first. The size of the image is relatively small compared to VMs. Other versions of the node image such as `node:slim` and `node:alpine` can give you even smaller images for easier portability. The topic of slimming down container sizes is further explored in Advanced Topics. You can view all versions in the official repository here.

Note, you can remove an image from your docker images using `docker rmi [repository]:[tag]`.

## Run a Docker container

Now we'll run the container based on the image you built above using the `docker run` command. The `--name` flag allows you to name the container if you like. And `-p` instructs Docker to map the host's port 4000 to the container's port 80. This allows you to reach the server at http://localhost:4000. Without port mapping, you would not be able to reach the container at localhost.

In [10]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                      PORTS     NAMES
74f7760c502c   hello-world                    "/hello"                 14 minutes ago   Exited (0) 13 minutes ago             awesome_morse
cc9930dc7af4   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   26 minutes ago   Up 24 minutes                         proxy-agent


**Exercise**

Use `docker run` to run the container you just build called `node-app:0.1`. Assign the host port `4000` to port `80` and assign it the name `my-app`.

In [15]:
%%bash
docker run -p 4000:80 --name my-app node-app:0.1

Server running at http://0.0.0.0:80/


CalledProcessError: Command 'b'docker run -p 4000:80 --name my-app node-app:0.1\n'' returned non-zero exit status 137.

To test out the server, open a terminal window and type the following command:

```bash
curl http://localhost:4000
```

You should see the server respond with `Hello World`

The container will run as long as the initial terminal is running. If you want to stop the container, run the following command in the terminal to stop and remove the container:

```bash
docker stop my-app && docker rm my-app
```
After a few moments the container will stop. You should notice the cell above will complete execution.

#### Running the container in the background
If you want to the container to run in the background (not tied to the terminal's session), you need to specify the `-d` flag.
Now run the following command to start the container in the background

**Exercise**

Modify your command above with `-d` flag to run `my-app` in the background.

In [18]:
%%bash
docker run -d -p 4000:80 --name my-app  node-app:0.1 

3e2b328cf3e6dbb67f829956303598dd80b16ca0f3cf7693a1939b70a442b364


Your container is now running in the background. You can check the status of your running container using `docker ps`

In [19]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS                  NAMES
3e2b328cf3e6   node-app:0.1                   "node app.js"            6 seconds ago    Up 5 seconds    0.0.0.0:4000->80/tcp   my-app
cc9930dc7af4   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   40 minutes ago   Up 38 minutes                          proxy-agent


Notice the container is running in the output of docker ps. You can look at the logs by executing `docker logs [container_id]`. 

In [20]:
# Note, your container id will be different
!docker logs 3e2b328cf3e6   

Server running at http://0.0.0.0:80/


You should see 
```bash
Server running at http://0.0.0.0:80/
```
If you want to follow the log's output as the container is running, use the `-f` option.

## Modify & Publish

Let's modify the application and push it to your Google Cloud Repository (gcr). After that you'll remove all local containers and images to simulate a fresh environment, and then pull and run your containers from gcr. This will demonstrate the portability of Docker containers.

### Edit `app.js`
Open the file `./src/app.js` with the text editor and replace "Hello World" with another string. Then build this new image. 

**Exercise**

After modifying the `app.js` file, use `docker build` to build a new container called `node-app:0.2` from the same docker file. 

In [21]:
%%bash
docker build -t node-app:0.2 ./  -f ./dockerfiles/intro.docker

Sending build context to Docker daemon  118.3kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> 8944fe56d3aa
Step 3/5 : COPY ./src/ /app
 ---> 3e57523a7c8b
Step 4/5 : EXPOSE 80
 ---> Running in c7f6f8add82f
Removing intermediate container c7f6f8add82f
 ---> 5bc9230a30fd
Step 5/5 : CMD ["node", "app.js"]
 ---> Running in 022ceaabe843
Removing intermediate container 022ceaabe843
 ---> 3f9b9d2a4916
Successfully built 3f9b9d2a4916
Successfully tagged node-app:0.2


Notice in `Step 2` of the output we are using an existing cache layer. From `Step 3` and on, the layers are modified because we made a change in `app.js`.

Run another container with the new image version. Notice how we map the host's port 8000 instead of 80. We can't use host port 4000 because it's already in use. 

**Exercise**

Run this new container in the background using a different port and with the name `my-app-2`.

In [23]:
%%bash
docker run -d -p 8000:80 --name my-app-2 node-app:0.2

5ab0371cde926b97c434110948a0186f128887d5f7ccb4b308126846a0447610


You can check that both container are running using `docker ps`.

In [24]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS                  NAMES
5ab0371cde92   node-app:0.2                   "node app.js"            5 seconds ago    Up 4 seconds    0.0.0.0:8000->80/tcp   my-app-2
3e2b328cf3e6   node-app:0.1                   "node app.js"            7 minutes ago    Up 7 minutes    0.0.0.0:4000->80/tcp   my-app
cc9930dc7af4   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   48 minutes ago   Up 46 minutes                          proxy-agent


And let's test boht containers using `curl` as before:

In [25]:
!curl http://localhost:8000

Just posting this altered message from gcr


In [26]:
!curl http://localhost:4000

Hello Wolrd!


Recall, to stop a container running, you can execute the following command either in a terminal or (because they are running in the background) in a cell in this notebook. 

### Publish to gcr

Now you're going to push your image to the Google Container Registry (gcr). To push images to your private registry hosted by gcr, you need to tag the images with a registry name. The format is `[hostname]/[project-id]/[image]:[tag]`.

For gcr:

  * `[hostname]`= gcr.io
  * `[project-id]`= your project's ID
  * `[image]`= your image name
  * `[tag]`= any string tag of your choice. If unspecified, it defaults to "latest".

In [27]:
import os

PROJECT_ID = "qwiklabs-gcp-04-28475851fd68"  # REPLACE WITH YOUR PROJECT NAME

os.environ["PROJECT_ID"] = PROJECT_ID

Let's tag `node-app:0.2`.

In [28]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED          SIZE
node-app                       0.2       3f9b9d2a4916   6 minutes ago    884MB
node-app                       0.1       81eb2923eadb   18 minutes ago   884MB
<none>                         <none>    13fc570556c6   24 minutes ago   884MB
hello-world                    latest    feb5d9fea6a5   6 months ago     13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   13 months ago    1.73GB
node                           6         ab290b853066   2 years ago      884MB


**Exercise**

Tag the `node-app:0.2` image with a new image name conforming to the naming convention `gcr.io/[project-id]/[image]:[tag]`. Keep the image and tag names the same.

In [29]:
%%bash
docker image tag node-app:0.2 gcr.io/$PROJECT_ID/node-app:0.2

Now when we list our docker images we should see this newly tagged repository.

In [30]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED          SIZE
node-app                                       0.2       3f9b9d2a4916   10 minutes ago   884MB
gcr.io/qwiklabs-gcp-04-28475851fd68/node-app   0.2       3f9b9d2a4916   10 minutes ago   884MB
node-app                                       0.1       81eb2923eadb   22 minutes ago   884MB
<none>                                         <none>    13fc570556c6   28 minutes ago   884MB
hello-world                                    latest    feb5d9fea6a5   6 months ago     13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   13 months ago    1.73GB
node                                           6         ab290b853066   2 years ago      884MB


Next, let's push this image to gcr.

**Exercise**

Push this new image to the gcr.

In [32]:
%%bash
docker push gcr.io/qwiklabs-gcp-04-28475851fd68/node-app:0.2

The push refers to repository [gcr.io/qwiklabs-gcp-04-28475851fd68/node-app]
12ebad091129: Preparing
40adcfcba6c3: Preparing
f39151891503: Preparing
f1965d3c206f: Preparing
a27518e43e49: Preparing
910d7fd9e23e: Preparing
4230ff7f2288: Preparing
2c719774c1e1: Preparing
ec62f19bb3aa: Preparing
f94641f1fe1f: Preparing
910d7fd9e23e: Waiting
4230ff7f2288: Waiting
2c719774c1e1: Waiting
ec62f19bb3aa: Waiting
f94641f1fe1f: Waiting
a27518e43e49: Layer already exists
f39151891503: Layer already exists
f1965d3c206f: Layer already exists
2c719774c1e1: Layer already exists
4230ff7f2288: Layer already exists
910d7fd9e23e: Layer already exists
ec62f19bb3aa: Layer already exists
f94641f1fe1f: Layer already exists
12ebad091129: Pushed
40adcfcba6c3: Pushed
0.2: digest: sha256:18f1caf49beff64a28a02334c9c4bd1aeaad679c22ca2d85525aea3b8efb55ad size: 2422


Check that the image exists in `gcr` by visiting the image registry Cloud Console. You can navigate via the console to `Navigation menu > Container Registry` or visit the url from the cell below:

In [33]:
%%bash
echo "http://gcr.io/${PROJECT_ID}/node-app"

http://gcr.io/qwiklabs-gcp-04-28475851fd68/node-app


### Test the published gcr image

Let's test this image. You could start a new VM, ssh into that VM, and install gcloud. For simplicity, we'll just remove all containers and images to simulate a fresh environment.

First, stop and remove all containers using `docker stop` and `docker rm`. **Be careful not to stop the container running this AI Notebook!**.

In [34]:
!docker stop my-app && docker rm my-app

my-app
my-app


In [35]:
!docker stop my-app-2 && docker rm my-app-2

my-app-2
my-app-2


Now remove the docker images you've created above using `docker rmi`.

In [36]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED          SIZE
gcr.io/qwiklabs-gcp-04-28475851fd68/node-app   0.2       3f9b9d2a4916   14 minutes ago   884MB
node-app                                       0.2       3f9b9d2a4916   14 minutes ago   884MB
node-app                                       0.1       81eb2923eadb   27 minutes ago   884MB
<none>                                         <none>    13fc570556c6   32 minutes ago   884MB
hello-world                                    latest    feb5d9fea6a5   6 months ago     13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   13 months ago    1.73GB
node                                           6         ab290b853066   2 years ago      884MB


In [37]:
%%bash
docker rmi node-app:0.2
docker rmi gcr.io/${PROJECT_ID}/node-app:0.2
docker rmi node-app:0.1
docker rmi node:6 
docker rmi -f hello-world:latest

Untagged: node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-04-28475851fd68/node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-04-28475851fd68/node-app@sha256:18f1caf49beff64a28a02334c9c4bd1aeaad679c22ca2d85525aea3b8efb55ad
Deleted: sha256:3f9b9d2a4916c8ed8647a706dd0a7075d74be29101543df6ce09eb265a718a51
Deleted: sha256:5bc9230a30fdfe11a4fb2a4d01b9235fbd1e22ad779e5a50145fcd63beb9be1a
Deleted: sha256:3e57523a7c8b77cd14cd188877b7da3487e2e3e4d6878eeb690b0285046a8410
Deleted: sha256:00a48ec1c28f6c5967d84936a69a13619ded2fa792f0dd1b406eec06a830bc55
Untagged: node-app:0.1
Deleted: sha256:81eb2923eadbe91de9f1fab670e1dc9b450da28ee9a3107d48b9c9a66cb89f8b
Deleted: sha256:d5ef84935c98896161d22806fc353b729572b7d31eb6a1f585d5f691545c3a10
Deleted: sha256:fe5c276c5549fbe5862aa1f4818684dfa6a787287833aabbd443f70af20b89a5
Deleted: sha256:91950f8b6a4e12cf2e02d1b7f7da3a11d656577d038a186a4b96618e4cb4f9e3
Untagged: node:6
Untagged: node@sha256:e133e66ec3bfc98da0440e552f452e5cdf6413319d27a2db3b01ac4b319759b3
Untagged: hello-

Confirm all images are removed with `docker images`.

In [38]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED          SIZE
<none>                         <none>    13fc570556c6   32 minutes ago   884MB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   13 months ago    1.73GB


At this point you should have a pseudo-fresh environment. Now, pull the image and run it.

In [39]:
%%bash
docker pull gcr.io/${PROJECT_ID}/node-app:0.2
docker run -p 4000:80 -d gcr.io/${PROJECT_ID}/node-app:0.2

0.2: Pulling from qwiklabs-gcp-04-28475851fd68/node-app
c5e155d5a1d1: Already exists
221d80d00ae9: Already exists
4250b3117dca: Already exists
3b7ca19181b2: Already exists
425d7b2a5bcc: Already exists
69df12c70287: Already exists
ea2f5386a42d: Already exists
d421d2b3c5eb: Already exists
599ff6aa77dc: Already exists
f496de6ce14e: Pulling fs layer
f496de6ce14e: Download complete
f496de6ce14e: Pull complete
Digest: sha256:18f1caf49beff64a28a02334c9c4bd1aeaad679c22ca2d85525aea3b8efb55ad
Status: Downloaded newer image for gcr.io/qwiklabs-gcp-04-28475851fd68/node-app:0.2
gcr.io/qwiklabs-gcp-04-28475851fd68/node-app:0.2
37d7b979c4cc3f255830782d9d39f47824bb903bf0af6ac18bb9a50281a3ad05


You can check that it's running as expected using before:

In [40]:
!curl http://localhost:4000

Just posting this altered message from gcr


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.